In [1]:
!pip install "Elasticsearch==8.7.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 387.9/387.9 kB 6.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 5.3 MB/s eta 0:00:00


In [ ]:
from elasticsearch import Elasticsearch

es = Elasticsearch("http://elasticsearch:8000")

# Verifica la conexión
print(es.info().body)

{'name': 'afb8a261363e', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'ed3nmv4bSAODb6aWxX5FuQ', 'version': {'number': '8.7.0', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '09520b59b6bc1057340b55750186466ea715e30e', 'build_date': '2023-03-27T16:31:09.816451435Z', 'build_snapshot': False, 'lucene_version': '9.5.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


In [ ]:
import pandas as pd

# Load CSV file
df = pd.read_csv("work/notebooks/grados.csv")
df.head()

,review_id,movie_title,review_text,rating,reviewer_name
0,1,The Pulp Fiction,It was decent. I recommend watching it if you ...,3,Clara
1,2,Avatar,An exciting and thrilling experience without a...,2,John
2,3,The Godfather,I found the movie quite underwhelming given th...,4,John
3,4,Pulp Fiction,This film is overrated. The plot is predictabl...,4,Alice
4,5,The Matrix,Too much focus on action; the characters were ...,2,Alice


In [ ]:
df.fillna({
        'nombre': 'Unknown', 
        'descripcion':'Este grado no contiene descripción',
        'salidas':'Este grado no contiene salidas'
        },
    inplace=True)

In [ ]:
df['id_area']=pd.to_numeric(df['rating'], errors='coerce')
df['id']=pd.to_numeric(df['review_id'], errors='coerce')
df.head()

,review_id,movie_title,review_text,rating,reviewer_name
0,1,The Pulp Fiction,It was decent. I recommend watching it if you ...,3,Clara
1,2,Avatar,An exciting and thrilling experience without a...,2,John
2,3,The Godfather,I found the movie quite underwhelming given th...,4,John
3,4,Pulp Fiction,This film is overrated. The plot is predictabl...,4,Alice
4,5,The Matrix,Too much focus on action; the characters were ...,2,Alice


In [ ]:
mapping = {
    "mappings": {
        "properties": {
            "id": { "type": "integer" },        
            "nombre": { "type": "keyword" },     
            "descripcion": { "type": "text" },
            "id_area": { "type": "integer" },
            "salidas": { "type": "text" },
        }
    }
}

index_name = 'salidas'
if es.indices.exists(index=index_name):
    es.indices.delete(index=index_name)

es.indices.create(index=index_name, mappings=mapping["mappings"])
print(f"Index '{index_name}' created with mapping.")

Index 'movie_reviews' created with mapping.


In [ ]:
import json
from elasticsearch.helpers import bulk

def bulk_index_data(es, data, index_name):
    batch_size = 50  # Reducir el tamaño del lote a 50
    for i in range(0, len(data), batch_size):
        batch = data[i:i+batch_size]
        actions = []
        for doc in batch:
            actions.append({
                "_index": index_name,
                "_id": doc['salidas'],
                "_source": doc
            })
        # Capturar la respuesta para verificar errores
        resp = bulk(es, actions, raise_on_error=True)
        print("Indexed:", resp[0], "Errors:", resp[1])

salidas = df.to_dict(orient='records')
bulk_index_data(es, salidas, index_name)


Indexed: 50 Errors: []
Indexed: 50 Errors: []
Indexed: 50 Errors: []
Indexed: 50 Errors: []
Indexed: 50 Errors: []
Indexed: 50 Errors: []
Indexed: 50 Errors: []
Indexed: 50 Errors: []
Indexed: 50 Errors: []
Indexed: 50 Errors: []
Indexed: 50 Errors: []
Indexed: 50 Errors: []
Indexed: 50 Errors: []
Indexed: 50 Errors: []
Indexed: 50 Errors: []
Indexed: 50 Errors: []
Indexed: 50 Errors: []
Indexed: 50 Errors: []
Indexed: 39 Errors: []


# --HASTA AQUÍ DEBERÍA FUNCIONAR EL SUBIRLO A ELASTICSEARCH (LO DE ABAJO NO)

In [17]:
import json

# Query 1

res = es.search(
    index="movie_reviews",
    size=3,
    query={
        "term": {
            "movie_title": "Inception"
        }
    }
)
print(json.dumps(res.body, indent=4))

{
    "took": 527,
    "timed_out": false,
    "_shards": {
        "total": 1,
        "successful": 1,
        "skipped": 0,
        "failed": 0
    },
    "hits": {
        "total": {
            "value": 94,
            "relation": "eq"
        },
        "max_score": 2.2972798,
        "hits": [
            {
                "_index": "movie_reviews",
                "_id": "14",
                "_score": 2.2972798,
                "_source": {
                    "review_id": 14,
                    "movie_title": "Inception",
                    "review_text": "An indie gem that deserves more attention for its innovative approach and storytelling.",
                    "rating": 10,
                    "reviewer_name": "David"
                }
            },
            {
                "_index": "movie_reviews",
                "_id": "47",
                "_score": 2.2972798,
                "_source": {
                    "review_id": 47,
                    "movie_title":

In [20]:
# Query

res = es.search(
    index="movie_reviews",
    size=3,
    query={
        "term": {
            "movie_text": "Amazing"
        }
    }
)
print(json.dumps(res.body, indent=4))

{
    "took": 7,
    "timed_out": false,
    "_shards": {
        "total": 1,
        "successful": 1,
        "skipped": 0,
        "failed": 0
    },
    "hits": {
        "total": {
            "value": 0,
            "relation": "eq"
        },
        "max_score": null,
        "hits": []
    }
}


In [21]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 1.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 3.8 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.1/797.1 MB 327.2 kB/s eta 0:00:0000:0100:21
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 1.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 1.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 1.0 MB/s eta 0:00:0000:0100:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 4.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:0000:0100:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 7.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━

In [24]:
# indice semántico

index_mapping = {
    "properties": {
        "content": { "type": "text" },
        "content_vector": {
            "type": "dense_vector",
            "dims": 3,  # Dimensionalityof the embeddings
            "index": True,
            "similarity": "cosine"
        }
    }
}

index_name = "vector_example"
if es.indices.exists(index=index_name):
    es.indices.delete(index=index_name)
es.indices.create(
    index=index_name,
    mappings=index_mapping
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'vector_example'})